# Multiple Dataset Harmonization Tutorial

In [1]:
0

0

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append("/home/icb/kemal.inecik/work/codes/idtrack")

In [17]:
import os
import pickle
import subprocess
import time

import idtrack
import pandas as pd
import numpy as np

Initialize the graph and pathfinder

In [5]:
local_dir = "/lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp"
test_results_dir = os.path.join(local_dir, "hlca_harmonization")
os.makedirs(test_results_dir, exist_ok=True)

The datasets of HLCA

In [6]:
base_path = "/lustre/groups/ml01/projects/2023_HLCA_LSikkema/HLCA_reproducibility/data"
dset0_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/ready/full")
dset1_dir = os.path.join(base_path, "HLCA_extended/extension_datasets/raw")

data_h5ad_dict = {
    "Meyer_2021": f"{dset0_dir}/meyer_2021.h5ad",
    "MeyerNikolic_unpubl": f"{dset0_dir}/meyer_nikolic_unpubl.h5ad",
    "Barbry_unpubl": f"{dset0_dir}/barbry.h5ad",
    "Thienpont_2018": f"{dset1_dir}/Lambrechts/lambrechts.h5ad",
    "Budinger_2020": f"{dset0_dir}/bharat.h5ad",
    "Banovich_Kropski_2020": f"{dset0_dir}/haberman.h5ad",
    "Sheppard_2020": f"{dset0_dir}/tsukui.h5ad",
    "Lambrechts_2021": f"{dset0_dir}/wouters.h5ad"
}

In [7]:
harmonizer = idtrack.HarmonizeFeatures(
    project_name="hlca",
    data_h5ad_dict=data_h5ad_dict,
    project_local_repository=test_results_dir,
    idtrack_local_repository=local_dir,
    target_ensembl_release=114,
    final_database="HGNC Symbol",
    organism_name="homo_sapiens",
    graph_last_ensembl_release=114
)

2025-06-02 14:50:52 INFO:verify_organism: Ensembl Rest API query to get the organism names and associated releases.
2025-06-02 14:50:55 INFO:graph_maker: The graph is being read: /lustre/groups/ml01/workspace/kemal.inecik/idtrack_temp/graph_homo_sapiens_min79_max114_narrow.pickle
2025-06-02 14:51:46 INFO:the_graph: Cached properties being calculated: combined_edges
2025-06-02 14:52:27 INFO:the_graph: Cached properties being calculated: combined_edges_assembly_specific_genes
2025-06-02 14:52:31 INFO:the_graph: Cached properties being calculated: combined_edges_genes
2025-06-02 14:53:07 INFO:the_graph: Cached properties being calculated: lower_chars_graph
2025-06-02 14:53:08 INFO:the_graph: Cached properties being calculated: get_active_ranges_of_id
2025-06-02 14:53:30 INFO:the_graph: Cached properties being calculated: available_external_databases
2025-06-02 14:53:31 INFO:the_graph: Cached properties being calculated: available_genome_assemblies
2025-06-02 14:53:32 INFO:the_graph: Cache

In [28]:
unified_adata = harmonizer.unify_multiple_anndatas(
    obs_columns_to_keep=None  # forget all obs columns.
)

100%|████████████████████████████████████| 8/8 [09:02<00:00, 67.75s/it, dataset=Lambrechts_2021, study_var=31595, union_var=33488, dbh=1943]


In [29]:
unified_adata

AnnData object with n_obs × n_vars = 879961 × 33488
    obs: 'handle_anndata'
    var: 'converted_id_Meyer_2021', 'converted_id_MeyerNikolic_unpubl', 'converted_id_Barbry_unpubl', 'converted_id_Thienpont_2018', 'converted_id_Budinger_2020', 'converted_id_Banovich_Kropski_2020', 'converted_id_Sheppard_2020', 'converted_id_Lambrechts_2021', 'converted_id', 'intersection'

In [30]:
unified_adata.obs

,handle_anndata
Meyer_2021_AAACCTGGTGTGAATA-1-WTDAtest7887999-0,Meyer_2021
Meyer_2021_AAACGGGCAACCGCCA-1-WTDAtest7887999-0,Meyer_2021
Meyer_2021_AACCATGGTACAGCAG-1-WTDAtest7887999-0,Meyer_2021
Meyer_2021_AACGTTGGTGTCCTCT-1-WTDAtest7887999-0,Meyer_2021
Meyer_2021_AACTCCCTCCTAGTGA-1-WTDAtest7887999-0,Meyer_2021
...,...
Lambrechts_2021_BAL029_GGGCATCTCTAACTCT-1,Lambrechts_2021
Lambrechts_2021_BAL029_TATGCCCTCATGGTCA-1,Lambrechts_2021
Lambrechts_2021_BAL029_TGCTGCTAGCGACGTA-1,Lambrechts_2021
Lambrechts_2021_BAL029_TTAGGACTCAGCAACT-1,Lambrechts_2021


In [31]:
unified_adata.var

,converted_id_Meyer_2021,converted_id_MeyerNikolic_unpubl,converted_id_Barbry_unpubl,converted_id_Thienpont_2018,converted_id_Budinger_2020,converted_id_Banovich_Kropski_2020,converted_id_Sheppard_2020,converted_id_Lambrechts_2021,converted_id,intersection
AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,AC016831.7,1
AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,AC019117.1,1
AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,AC019117.2,1
AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,AC058791.1,1
AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,AC062029.1,1
...,...,...,...,...,...,...,...,...,...,...
MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,MTRNR2L3,1
MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,MTRNR2L4,1
PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,PRR34-AS1,1
SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,SMC5-AS1,1


In [32]:
np.sum(unified_adata.var["intersection"])

13799

In [33]:
1

1